# Проверка гипотезы по автоатизации проверки данных DataSet на предмет "подозрительных" выбросов занчений данных

 Мы провели графический анализ данных (см. Diploma_1). Исследование  данных в столбцах по типу box-and-wisker plot (ящик с усами) наглядно поазывают ,что в ряде столбцов ,таких как:
Шаг нашивки                       
Плотность нашивки                 
модуль упругости, ГПа            
Количество отвердителя, м.%     
Поверхностная плотность, г/м2   
Потребление смолы, г/м2
имеются выбросы как в минимальной области (даже включя ноль, либо значения близкие к нулю - что противоречит физическим принципам)
Либо значения на порядок, а то и на 3 поядка меньше медианы данных конкретного столбца так  и в максимальной области.

Для принятия решения по изменению конкретных данных (замена на медиану или на умножение на конкретный коэффициент) или удалению конкретной строки данных в случае сильной флуктуации данных в строке принимаем гипотезу по созданию матрицы "подозрительных данных" для дальнейшей аналиа.

In [ ]:
# Импортируем в проект требуемые библиотеки

import seaborn as sns # библиотека для создания статистических графиков
import random #  генераторатор случайных чисел и данных
import pandas as pd # библиотека для обработки и анализа данных
import os # библиотека функций для работы с операционной системой.
import numpy as np
import tensorflow as tf # фреймворк для глубокого машинного обучения
import csv
#import matplotlib.pyplot as plt
#from matplotlib import cm
#from mpl_toolkits.mplot3d import Axes3D
#import matplotlib.pyplot as plt

#import plotly
#import plotly.graph_objs as go
#import plotly.express as px
#from plotly.subplots import make_subplots

# Создадим матрицу(массив) "подозрительных данных"

In [ ]:
# теперь прочитаем наш файл формата hw_data_composite_new.CSV в  котором будем изменять часть данных в определенных строках

hw_data_composite_analyzis = pd.read_csv(r'C:\Users\grain\Work_folder\Diplom_MGTU\hw_data_composite_new.csv')
hw_data_composite_analyzis

In [ ]:
# применим метод .describe() к количественным признакам
hw_data_composite_analyzis.describe(percentiles = [0.25, 0.5, 0.75, 0.99], include='all').round(3) # дополнительно выводим еще одну квартилю 99%

In [ ]:
hw_data_composite_analyzis.head(21)

In [ ]:
hw_data_composite_analyzis.loc[20][1]

# замечание - в  строке с индексом 19 в столбце 'Шаг нашивки' и 'Плотность нашивки' есть значения '0.0'.
нам необходимо провести анализ и найти 'подозрительного значения'

# Заметим, что '50% процентиль' тоже, что и медиана!!!
Источник - https://pandas.pydata.org/docs/dev/reference/api/pandas.DataFrame.describe.html

In [ ]:
# Вычислим список медиан по всем столбцам - это процентиль 50%

median_composit = hw_data_composite_analyzis.describe(include='all').loc['50%']
median_composit

# Создадим DataFrame 'median_set' из названий столбцов и значений медиан столбцов
# Источник - https://stackoverflow.com/questions/26097916/convert-pandas-series-to-dataframe

#median_set = pd.DataFrame({'Сolumns':median_composit.index, '50%-median ':median_composit.values})
#median_set # выводим на печать

# df_zeros = pd.DataFrame(hw_data_composite_analyzis.applymap(lambda x: 0.0))  # создаем df заполненый 'o.o'  с данными типа float64
#df_zeros = df_zeros.astype(np.float32)               # меняем тип данных  массива на float32
#df_zeros.tail()

# new_row_median = hw_data_composite_analyzis.median(axis=0) # параметры новой строки в конце массива - это медианы столбцов!!

# df_zeros.append(new_row_median, ignore_index = True) # создаем новую строку медиан в конце массива # работает , строка добавляется



# значение критериев 'min_data'  - все значения меньше 10% от медианы- 'подозрительного значения'
# значение критериев 'max_data'  - все значения больше медианы в 10 раз  - 'подозрительного значения'
min_data = 0.1
max_data = 10.0
 
# переводим список медиан в DataFrame
# df_median = pd.DataFrame({'Сolumns':median_composit.index, '50%-median ':median_composit.values}) 

for column in hw_data_composite_analyzis.columns: #
    for i in range(len(hw_data_composite_analyzis)):
        
        value = hw_data_composite_analyzis.at[i,column] # значение из df -  hw_data_composite_analyzis ВАЖНО присвоение .at[i,column]
        min_median = min_data * median_composit[column] # значение критерия 'min_median' = критерий min * медиану столбца
        max_median = max_data * median_composit[column] # значение критерия 'mvax_median' = критерий max * медиану столбца
               
        if value < min_median  or value > max_median: # условие применения критериев
            df_zeros.at[i,column] = value  # запись 'подозрительного значения' в соотвествующую ячейку нолевой матрицы df_zeros
            

# Создадим строку медиан из hw_data_composite_analyzis и добавим ее в конце массива df_zeros!!!#new_row_median = hw_data_composite_analyzis.median(axis=0) # параметры новой строк в конце массива!!
#df_zeros.append(new_row_median, ignore_index = True) # создаем новую строку медиан

# df2 = pd.DataFrame([[2,3,4]], columns=['A','B','C']) # Другой метод добавления строки создание df медианн и concat! 
# pd.concat([df2, df])

In [ ]:
def analysis_dataset(df, min_data, max_data, median_composit):
# Функция analyzis_dataset создает массив 'подозрительных значений' в матрице заполненой нулями df_zeros
# Передем в функцию:
# df который будем исследовать
# min_data - минимальный коэффициент отсечки для выбросов значений "слева"
# max_data - максимальный коэффициент отсечки для выбросов значений "справа"
# значение критериев 'min_data'  - все значения меньше 10% от медианы- 'подозрительного значения'
# значение критериев 'max_data'  - все значения больше медианы в 10 раз  - 'подозрительного значения'
# принимаем min_data = 0.1
#           max_data = 10.0
# median_composit - список median массива df рассчитан ранее в программе

# создаем df_zero заполненый 'o.o'  размерности  df ( по умолчанию - с данными типа float64)
    df_zeros = pd.DataFrame(df.applymap(lambda x: 0.0)) 
    df_zeros = df_zeros.astype(np.float32)               # меняем тип данных  массива на float32

    for column in df.columns: # иттерация по столбцам
        for i in range(len(df)): # иттерация по строкам
            
            value = df.at[i,column] # значение из df -  hw_data_composite_analyzis ВАЖНО писвоение .at[i,column]
        
            min_median = min_data * median_composit[column] # значение критерия 'min_median' = критерий min * медиану столбца
            max_median = max_data * median_composit[column] # значение критерия 'mvax_median' = критерий max * медиану столбца
               
            if value < min_median  or value > max_median: # условие применения критериев
                
                df_zeros.at[i,column] = value  # запись 'подозрительного значения' в соотвествующую ячейку нолевой матрицы df_zeros  
                
                if value == 0.0:
                    df_zeros.at[i,column] = 0.1

    return df_zeros

# Необходимо создать  отдельный модуль - функцию для проверки нашего DataSet 'hw_data_composite_analyzis' на 'подозрительные значения' и создание массива 'подозрительных значений'

# проверка функции

In [ ]:
df_zeros = analysis_dataset(hw_data_composite_analyzis, 0.1, 10, median_composit)  

In [ ]:
df_zeros.info()

In [ ]:
df_zeros = df_zeros.astype(np.float32) # опять приводим размерность данных к типу float32

In [ ]:
df_zeros.info()

In [ ]:
df_zeros.head(21).round(3)

# Рассчитаем количество строк в которых есть 'подозрительные значения' в массиве df_zeros.
Создадим массив строк df_analiz используя массив df_zeros, сохранив их оригинальные индексы в отдельном столбце и введя особый столбец с указнием количества 'подозрительных значений'
Если в какой-то строке будет больше 1 'подозрительноо значения', то  скорее всего мы будем удалять эту строку в нашем рабочем файле hw_data_composite_analyzis

In [ ]:
# создадим новый  df_analiz_id для вывода индексов строк  'подозрительных значений'  с одной строкой с нулевыми значениями 
# (далее будем добавлять строки по мере возникнвения новых строк с ощибками) .

df_analis_id = pd.DataFrame(columns=('Индекс строки исходного DataSet', 'Колиества ошибок в строке'))
#df_analis = pd.DataFrame(columns=df_zeros.columns) # передали в новый df наиенования старых толбцов.

# теперь добавим в df_analiz 2 новых  столбца: 'Индекс старого массива', 'Колиества ошибок в строке'
#df_analis.insert (loc = 0, column='Колиества ошибок в строке', value= False) # вставляем певую колонку вначале таблицы
#df_analis.insert (loc = 0, column='Индекс старого массива', value= False) # вставляем  колонку на первое место
#df_analis.columns

In [ ]:
df_analis_id.shape #  у нас получилась таблица на 2 столбцов и пока 0 строки

In [ ]:
df_analis_id

In [ ]:
df_analis_id.info()

In [ ]:
df_analis_bugs = pd.DataFrame(columns=df_zeros.columns) # передали в созданый новый df наиенования старых толбцов, но у нс 0 строк


In [ ]:
df_analis_bugs.shape

In [ ]:
df_analis_bugs.info()

# Добавим строку  состоящую из 0.0
df_analis_bugs.loc[len(df_analis_bugs.index)]= [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 

In [ ]:
#df_analis_bugs.info()

In [ ]:
df_analis_bugs

#df_analis_bugs = df_analis_bugs.astype(np.float32) # приводим размерность данных к типу float32
#df_analis_bugs.info()

ПРОВЕРИТЬ!!!
# df_analis_id = df_analis_id.astype(np.Int32) # приводим размерность данных к типу int32
# df_analis_id.info()

In [ ]:
# Подсчитаем в каждой строке матрицы количество 'подозрительных значений' - df_zeros и будем записывать
# в создаваемую строку j  матрицы df_analis_bugs:
# а также запишем в два  столбца df_analis_id в создаваемую строку j получаемые результаты:
# индекс строки  cтарого массива df_zeros, количества ошибок coun_bug_row в соотвествующем столбце

new_id = 0             # индекс первой строк массива df_analis_id и df_analis_bugs

for i in range(len(df_zeros)):      # len(df_zeros) -это количество строк -1023
    
    count_bug_row = 0               # количества ошибок count_bug_row в строке 'i' массива df_zeros
    
    
    for column in df_zeros.columns:
      
        if df_zeros.loc[i, column] != 0.0 and count_bug_row == 0:  # .at
            
            # Добавим строку  состоящую из 0.0
            df_analis_bugs.loc[len(df_analis_bugs.index)]= [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
            new_row = {'Индекс строки исходного DataSet':[0],'Колиества ошибок в строке':[0]}
            df_analis_id =  df_analis_id.append(new_row, ignore_index=True)
        df = pd.concat([df, new_df], axis=0, ignore_index=True)
       
    
        elif  df_zeros.loc[i, column] != 0.0:    
            print('new_id', new_id, 'i  ', i, 'column', column)  # работает 
            print('------' )           
            df_analis_id.at[new_id, 0] = i  # работает !! # записываем в столбец "Индекс старого массива" номер старого индекса строки, чтобы можно было найти в основном файлене !!нужено ПРОВЕРИТЬ!!!    
            
            print(df_analis_id.at[new_id, 0]) # получается float??? работает
            
            value1 = df_zeros.at[i,column] # записываем значение ячейки в переменную value1 РАБОТАЕТ
            print('value1',df_zeros.at[i,column])
            print('Мммм------' )     
            df_analis_bugs.at[new_id, column] = value1 #  записываем 'подозрительное значение' в cотвествующий столбец параметра 
            
            print('номер нового индекса new_id = ', new_id, end='\n')
            print('номер старого индекса  i =', i, column)
            print('ЦЦЦЦЦЦ------' )
  
            count_bug_row = count_bug_row + 1 #.at
            df_analis_id.loc[new_id, 1] = count_bug_row # записываем колчество 'подозрительных значений' в столбец 'количество bugs' старого индекса  i = проверяемой строке
            print('количество bugs',  df_analis_id.loc[new_id, 1])
            print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX') 
    #print('Конец цикла XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')     
        # добавляем новую строку в df_analis_bugs и df_analis_id
   
   # df_analis_id.loc[len(df_analis_id.index)] = [0.0, 0.0]
    
        #print('количество bugs', count_bug_row, end='\n')
        #print('________________________')
    
   # j = j + 1        
           
# df_analis.info()


In [ ]:
df_analis_bugs

In [ ]:
df_analis_id

In [ ]:
temp_df = temp_df.drop_duplicates() убираем дубликаты если есть, но сначала делаем сопию файла и уже убираем дубликаты из копии, не затрагивая основной файл

Использование inplace=True изменит исходный объект DataFrame:
Еще один важный аргумент drop_duplicates() — keep, который имеет три возможных опции:

first: Отбросить дубликаты, кроме первого вхождения (опция по умолчанию).
last: Отбросить дубликаты, кроме последнего вхождения.
False: Отбросить все дубликаты.
Поскольку в предыдущем примере мы не определили аргумент keep, он был задан по умолчанию как first. Это означает, что если две строки одинаковы, pandas отбросит вторую и сохранит первую. Использование last имеет противоположный эффект: отбрасывается первая строка.

А вот keep=False отбрасывает все дубликаты. Если две строки одинаковы, то обе будут отброшены. Посмотрите, что произойдет с temp_df:

Мы можем использовать метод .rename() для переименования определенных или всех столбцов с помощью словаря. Нам не нужны круглые скобки, поэтому давайте избавимся от них:

!!!Если вы будете работать с набором данных в течение некоторого времени, рекомендуется использовать нижний регистр, удалить специальные символы и заменить пробелы символами подчеркивания.